<a href="https://colab.research.google.com/github/tanishkamaurya/Real-Time-Parking-Pricing/blob/main/Capstone_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pathway bokeh panel --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install google-cloud-bigquery --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.6/253.6 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.29.0
    Uninstalling google-cloud-bigquery-3.29.0:
      Successfully uninstalled google-cloud-bigquery-3.29.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathway 0.24.1 requires google-cloud-bigquery~=3.29.0, but you have google-cloud-bigquery 3.34.0 which is incompatible.


In [3]:
!pip install pathway --upgrade

  Using cached google_cloud_bigquery-3.29.0-py2.py3-none-any.whl.metadata (7.6 kB)
Using cached google_cloud_bigquery-3.29.0-py2.py3-none-any.whl (244 kB)
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.34.0
    Uninstalling google-cloud-bigquery-3.34.0:
      Successfully uninstalled google-cloud-bigquery-3.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.29.0 which is incompatible.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
# from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()


In [5]:
from google.colab import drive
file_id = '1RqHF3zphAFOtYZgReDJUxEFweOiVAxqP'
url = f'https://drive.google.com/uc?id={file_id}'

df = pd.read_csv(url)
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [6]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))


In [7]:
# This cell is no longer needed as the data is loaded in cell yzdfFDdQssbL
# filename = "dataset.csv"
# df = pd.read_csv(f'https://drive.google.com/file/d/1RqHF3zphAFOtYZgReDJUxEFweOiVAxqP/view?usp=drive_link')
# df.to_csv(filename, index=False)

In [8]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

df.rename(columns={'SystemCodeNumber': 'LocationID'}, inplace=True)

df[['Timestamp', 'LocationID', 'Occupancy', 'Capacity', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']].to_csv('parking_stream.csv', index=False)

In [9]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    LocationID: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str # Changed to string based on the original dataframe
    IsSpecialDay: int # Changed to int based on the original dataframe
    VehicleType: str

is_special_day = pw.this.IsSpecialDay * 1.0


In [10]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


MODEL 1 : BASELINE LINEAR MODEL

In [11]:
import datetime

baseline_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.LocationID),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_sum = pw.reducers.sum(pw.this.Occupancy),
        occ_count = pw.reducers.count(),
        cap = pw.reducers.max(pw.this.Capacity),
        location = pw.reducers.any(pw.this.LocationID)
    )
    .with_columns(
        occupancy_rate = (pw.this.occ_sum / pw.this.occ_count) / pw.this.cap,
        price = 10.0 + 5.0 * ((pw.this.occ_sum / pw.this.occ_count) / pw.this.cap)
    )
)


MODEL 2: DEMAND BASED PRICE FUNCTION

In [12]:
import datetime
import pathway as pw

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.LocationID),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_sum = pw.reducers.sum(pw.this.Occupancy),
        occ_count = pw.reducers.count(),
        cap = pw.reducers.max(pw.this.Capacity),
        queue_sum = pw.reducers.sum(pw.this.QueueLength),
        traffic_high = pw.reducers.count(pw.this.TrafficConditionNearby == "high"),
        traffic_medium = pw.reducers.count(pw.this.TrafficConditionNearby == "medium"),
        traffic_low = pw.reducers.count(pw.this.TrafficConditionNearby == "low"),
        special_day_count = pw.reducers.sum(pw.this.IsSpecialDay),
        vehicle_car = pw.reducers.count(pw.this.VehicleType == "car"),
        vehicle_bike = pw.reducers.count(pw.this.VehicleType == "bike"),
        vehicle_truck = pw.reducers.count(pw.this.VehicleType == "truck"),
        location = pw.reducers.any(pw.this.LocationID),
    )
    # Step 1: Calculate demand_score
.with_columns(
    demand_score = (
        0.4 * ((pw.this.occ_sum / pw.this.occ_count) / pw.this.cap) +
        0.2 * (pw.this.queue_sum / pw.this.occ_count) +
        0.2 * (
            (1.5 * pw.this.traffic_high +
             1.0 * pw.this.traffic_medium +
             0.5 * pw.this.traffic_low) / pw.this.occ_count
        ) +
        0.1 * (pw.this.special_day_count / pw.this.occ_count) +
        0.1 * (
            (1.0 * pw.this.vehicle_car +
             0.5 * pw.this.vehicle_bike +
             1.5 * pw.this.vehicle_truck) / pw.this.occ_count
        )
    )
)

# Step 2: Normalize demand_score
    .with_columns(
    normalized_demand = pw.if_else(
        (pw.this.demand_score / 5.0) < 0.0, 0.0,
        pw.if_else(
            (pw.this.demand_score / 5.0) > 1.0, 1.0,
            pw.this.demand_score / 5.0
        )
    )
)


# Step 3: Compute final price
.with_columns(
    price = 10.0 + 10.0 * pw.this.normalized_demand
))


In [13]:
df[['Timestamp', 'LocationID', 'Occupancy', 'Capacity',
    'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']].to_csv('parking_stream.csv', index=False)


In [14]:
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig


In [23]:
viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [24]:
%%capture --no-display
pw.run()


Output()

KeyboardInterrupt: 

In [17]:
delta_window.schema.columns()

mappingproxy({'t': ColumnSchema(dtype=DATE_TIME_NAIVE, name='t', primary_key=False, default_value=undefined, append_only=False, description=None, example=None),
              'occ_sum': ColumnSchema(dtype=INT, name='occ_sum', primary_key=False, default_value=undefined, append_only=False, description=None, example=None),
              'occ_count': ColumnSchema(dtype=INT, name='occ_count', primary_key=False, default_value=undefined, append_only=False, description=None, example=None),
              'cap': ColumnSchema(dtype=INT, name='cap', primary_key=False, default_value=undefined, append_only=False, description=None, example=None),
              'queue_sum': ColumnSchema(dtype=INT, name='queue_sum', primary_key=False, default_value=undefined, append_only=False, description=None, example=None),
              'traffic_high': ColumnSchema(dtype=INT, name='traffic_high', primary_key=False, default_value=undefined, append_only=False, description=None, example=None),
              'traffic_m

In [18]:
df = df.rename(columns={"LocationID": "SystemCodeNumber"})

# Step 1: Clean and save
df.columns = df.columns.str.strip().str.replace("'", "")
df.to_csv("lot_info.csv", index=False)

class LotSchema(pw.Schema):
    SystemCodeNumber: str
    Latitude: float
    Longitude: float

lot_info = pw.io.csv.read("lot_info.csv", schema=LotSchema)
# lot_info = pw.io.python.read(
#     df[["SystemCodeNumber", "Latitude", "Longitude"]].to_dict(orient="records"),
#     schema=LotSchema



In [19]:
print(df.columns.tolist())


['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'Timestamp']


In [20]:
# Step 1: Create lot_a and lot_b for self-join
lot_a = lot_info.select(
    SystemCodeA = pw.this.SystemCodeNumber,
    LatA = pw.this.Latitude,
    LonA = pw.this.Longitude
)

lot_b = lot_info.select(
    SystemCodeB = pw.this.SystemCodeNumber,
    LatB = pw.this.Latitude,
    LonB = pw.this.Longitude
)

# Perform join
lot_pairs = pw.join(lot_a, lot_b).filter(
    pw.left.SystemCodeA != pw.right.SystemCodeB
)


In [21]:
lot_pairs_flat = lot_pairs.select(
    SystemCodeA = pw.left.SystemCodeA,
    LatA = pw.left.LatA,
    LonA = pw.left.LonA,
    SystemCodeB = pw.right.SystemCodeB,
    LatB = pw.right.LatB,
    LonB = pw.right.LonB
)

In [22]:
df["distance"] = df.apply(lambda row: haversine(row["LatA"], row["LonA"], row["LatB"], row["LonB"]), axis=1)


KeyError: 'LatA'